<a href="https://colab.research.google.com/github/alcliberal/multirotor_sizing_isae_coa_2021/blob/main/05c_BatteryESCSelection_Student_Melissabis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://github.com/alcliberal/multirotor_sizing_isae_coa_2021/blob/main/pictures/logo_sizinglab.png?raw=1" style="float:right; max-width: 60px; display: inline" alt="SizingLab" /></a>

# Battery and ESC selection
*Written by Marc Budinger (INSA Toulouse) and Scott Delbecq (ISAE-SUPAERO), Toulouse, France.*

## Design graph 

The following diagram represents the design graph of the battery and ESC selection.


![DesignGraph](https://github.com/alcliberal/multirotor_sizing_isae_coa_2021/blob/main/pictures/ESCBatteryDesignGraph.png?raw=1)

> **Questions:**
* Recall the main equations to take into account during ESC and battery selection
* Orientate the arrows
* Give equations order, inputs/outputs at each step of this part of sizing procedure

*   Main equations to take into account during ESC and battery selection :
    - M_esc (kg) = 2.88e-05 * Pesc   (w) (according to notebook 3)
    - Pesc > Pmax (take-off)
    - t (h) = C(Ah) *Vbat (V) / Pmoy(W)
    - Vbat > U_mot_to
    - V_bat = V_cell * ns
    - Ebat (Wh) =  M_bat * E_massique (220 Wh/kg)


*   Oriented diagram



*   Sizing procedure
    - Pesc = kesc * P_esc_ref with 1 <= kesc <= 10 
    - M_esc = 0.02876598 * Pesc
    - Vbat =  U_bat_estimé * kmot    1 <= kmot <= 10 
    - C (Ah) = Pmoy * t / Vbat
    - C (Wh) = Pmoy* t
    - M_bat =  C (Wh) /E_massique
    - E_massique (?? fonction de ns ??)



### Sizing code and optimization

> Exercice: propose a sizing code for the selection of a battery and ESC.


In [ ]:
# Specifications
N_pro=4.0#[-] Number of propellers
M_pay=1.0#[kg] Payload mass
t_hov = 3 #[h]

# Reference parameters for scaling laws
# Ref : MK-quadro
M_bat_ref = .329 # [kg] mass
E_bat_ref = 220.*3600.*.329 # [J]

# Ref : Turnigy K_Force 70HV 
P_esc_ref = 3108. # [W] Power
M_esc_ref = .115 # [kg] Mass

# Assumption
U_bat_est=14.0#[V] Battery voltage estimation
P_el_mot_hov=10.0#[W] Electrical power consumption for one motor during hover
P_el_mot_to=30.0#[W] Electrical power consumption for one motor during takeoff
U_mot_to=12.0#[V] Motor voltage during takeoff



In [ ]:
# Battery selection & scaling laws sized from hover
    
    
    U_bat = 1.84e+00*P_el_mot_to**(3.60e-01) # [V] Battery voltage
    N_s_bat = ceil(U_bat/3.7)  # [-] Cell number, round (up value).  from math import *

    M_bat = k_mb * M_pay # [kg] Battery mass


# Hover --> autonomy
    E_bat = E_bat_ref * M_bat_ref / M_bat * 0.8 # [J] Energy  of the battery (.8 coefficient because 80% use only of the total capacity)
    C_bat = E_bat/U_bat  # [A.s] Capacity  of the battery 
    I_bat =  P_el_mot_hov * N_pro / (0.95 * U_bat)  # [A] Current of the battery   #notebook 2
    t_hov = C_bat /I_bat*60  # [min] Hover time 


 #ESC    
  
    P_esc = P_el_mot_to # [W] power electronic power (corner power or apparent power)
    M_esc =  2.88e-05 * Pesc  # [kg] Mass ESC
    V_esc =  1.84e+00*P_esc^(3.60e-01) # [V] ESC voltage  

In [ ]:
#Design variables

# -----------------------
# equations
# -----------------------

# Hover --> autonomy

# ESC


#Design variables
# kesc from 1 to 10
# -----------------------
# equations

import pandas as pd

import ipywidgets as widgets
from ipywidgets import interactive

pd.options.display.float_format = '{:,.2f}'.format


# -----------------------
# sizing code
# -----------------------
# inputs: 
# - param: optimisation variables vector (reduction ratio, oversizing coefficient)
# - arg: selection of output  
# output: 
# - objective if arg='Obj', problem characteristics if arg='Prt', constraints other else

def SizingCode(param, arg):
# Design variables in param
# ---
    kesc = param[0] # variable x
     
# Calculus 
# ---
    P_esc = kesc * P_esc_ref
    M_esc = 2.88e-02 * P_esc
        
# Objective and Constraints sum up
# ---
    objective = M_esc
    
    constraints = [P_esc - P_el_mot_to,
                  ]
    
# Returns selection
# -------------------
    if arg == 'Obj':
        return objective

    elif arg=='Prt':
    # the data to print a defined into a Pandas dataframe
        col_names = ['Type', 'Name', 'Value', 'Unit', 'Comment']

        df = pd.DataFrame()
    
      
        df = df.append([{'Type': 'Objective', 'Name': 'M_esc', 'Value': objective, 'Unit': '[g]', 'Comment': 'Mini mass ESC'}])[col_names]
        df = df.append([{'Type': 'Constraints', 'Name': 'P_esc > P_max', 'Value': constraints[0], 'Unit': '[W]', 'Comment': 'Power Constraint'}])[col_names]
        df = df.append([{'Type': 'Variables', 'Name': 'kesc', 'Value': kesc, 'Unit': '[-]', 'Comment': 'kesc'}])[col_names]
        df = df.append([{'Type': 'ESC', 'Name': 'P_esc', 'Value': P_esc, 'Unit': '[W]', 'Comment': 'P_esc'}])[col_names]

    # the dataframe is then organised to be printed with interactive widgets
        items = sorted(df['Type'].unique().tolist())

        def f(Type):
            return df[df['Type']==Type] 
        widgets.interact(f, Type=items)
        return f
        
    else:
        return constraints

In [ ]:
import scipy
import scipy.optimize

# Vector of initial parameters
parameters = scipy.array((1))

# Optimization with SLSQP algorithm: definition of requested functions
contrainte = lambda kesc: SizingCode(kesc, 'Const')
objectif = lambda kesc: SizingCode(kesc, 'Obj')
objectifP = lambda kesc: SizingCode(kesc, 'ObjP')

# Optimization bounds
bounds = [(0.5,5)]

# SLSQP omptimisation
result = scipy.optimize.fmin_slsqp(func=objectif, x0=parameters, 
                                   bounds=bounds,
                                   f_ieqcons=contrainte, iter=1500, acc=1e-12)


# Final characteristics after optimization 
print("-----------------------------------------------")
print("Final characteristics after optimization :")

print(SizingCode(result, 'Obj'))
SizingCode(result, 'Prt')

Optimization terminated successfully.    (Exit mode 0)
            Current function value: 44.75520001382431
            Iterations: 6
            Function evaluations: 6
            Gradient evaluations: 2
-----------------------------------------------
Final characteristics after optimization :
44.75520001382431


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: scipy.array is deprecated and will be removed in SciPy 2.0.0, use numpy.array instead
  """


interactive(children=(Dropdown(description='Type', options=('Constraints', 'ESC', 'Objective', 'Variables'), v…

<function __main__.SizingCode.<locals>.f>

In [ ]:
# Hover --> autonomy

import pandas as pd

import ipywidgets as widgets
from ipywidgets import interactive

pd.options.display.float_format = '{:,.2f}'.format


# -----------------------
# sizing code
# -----------------------
# inputs: 
# - param: optimisation variables vector (reduction ratio, oversizing coefficient)
# - arg: selection of output  
# output: 
# - objective if arg='Obj', problem characteristics if arg='Prt', constraints other else

def SizingCode(param, arg):
# Design variables in param
# ---
    kmot = param[0] # variable x
    
# Calculus 
# ---
    V_bat = U_bat_est * kmot
    C = P_el_mot_hov * t_hov
    M_bat = C /220
    E_bat = 220.*3600.*M_bat
        
# Objective and Constraints sum up
# ---
    objective = M_bat
    
    constraints = [V_bat - U_mot_to]
    
# Returns selection
# -------------------
    if arg == 'Obj':
        return objective

    elif arg=='Prt':
    # the data to print a defined into a Pandas dataframe
        col_names = ['Type', 'Name', 'Value', 'Unit', 'Comment']

        df = pd.DataFrame()
    
      
        df = df.append([{'Type': 'Objective', 'Name': 'M_bat', 'Value': objective, 'Unit': '[Kg]', 'Comment': 'Mini mass battery'}])[col_names]
        df = df.append([{'Type': 'Constraints', 'Name': 'Power', 'Value': constraints[0], 'Unit': '[W]', 'Comment': 'Power constraint'}])[col_names]
        df = df.append([{'Type': 'Variables', 'Name': 'kmot', 'Value': kmot, 'Unit': '[]', 'Comment': 'kmot'}])[col_names]
        df = df.append([{'Type': 'Battery', 'Name': 'Capacity', 'Value': C, 'Unit': '[Wh]', 'Comment': 'Battery capacity'}])[col_names]
        df = df.append([{'Type': 'Battery', 'Name': ' Voltage', 'Value': V_bat, 'Unit': '[V]', 'Comment': 'Battery voltage'}])[col_names]
        df = df.append([{'Type': 'Battery', 'Name': ' Energy', 'Value': E_bat, 'Unit': '[J]', 'Comment': 'Battery energy'}])[col_names]

    # the dataframe is then organised to be printed with interactive widgets
        items = sorted(df['Type'].unique().tolist())

        def f(Type):
            return df[df['Type']==Type] 
        widgets.interact(f, Type=items)
        return f
        
    else:
        return constraints

In [ ]:
import scipy
import scipy.optimize

# Vector of initial parameters
parameters = scipy.array((1))

# Optimization with SLSQP algorithm: definition of requested functions
contrainte = lambda kmot: SizingCode(kmot, 'Const')
objectif = lambda kmot: SizingCode(kmot, 'Obj')
objectifP = lambda kmot: SizingCode(kmot, 'ObjP')

# Optimization bounds
bounds = [(1,10)]

# SLSQP omptimisation
result = scipy.optimize.fmin_slsqp(func=objectif, x0=parameters, 
                                   bounds=bounds,
                                   f_ieqcons=contrainte, iter=1500, acc=1e-12)


# Final characteristics after optimization 
print("-----------------------------------------------")
print("Final characteristics after optimization :")

print(SizingCode(result, 'Obj'))
SizingCode(result, 'Prt')

Optimization terminated successfully.    (Exit mode 0)
            Current function value: 0.13636363636363635
            Iterations: 1
            Function evaluations: 3
            Gradient evaluations: 1
-----------------------------------------------
Final characteristics after optimization :
0.13636363636363635


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: scipy.array is deprecated and will be removed in SciPy 2.0.0, use numpy.array instead
  """


interactive(children=(Dropdown(description='Type', options=('Battery', 'Constraints', 'Objective', 'Variables'…

<function __main__.SizingCode.<locals>.f>